Juan Mata Naranjo 1939671

Stefano D'Arrigo 1960500

Daniel Jimenez Gutiérrez 1939216


In [2]:
#connect google drive to colab
from google.colab import drive
drive.mount('/content/gdrive/', force_remount = True)

Mounted at /content/gdrive/


## Install Packages

In [4]:
!pip install Node2Vec
!pip install Word2Vec

  Created wheel for Node2Vec: filename=node2vec-0.4.3-cp37-none-any.whl size=5982 sha256=cbcc61398763aabbb38d664d4a11f66e9cd5228c6ea1f3c64cdca4d1703d905c
  Stored in directory: /root/.cache/pip/wheels/34/f4/60/55708b5e797260310451d919c8f898bbd4acd1901367001f20
Successfully built Node2Vec
     |████████████████████████████████| 51kB 3.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for Word2Vec: filename=word2vec-0.11.1-cp37-none-any.whl size=156422 sha256=8c2f86c09471b0254b2cced4e975bc4e7ec0d568d9260ddade804b36b08841d4
  Stored in directory: /root/.cache/pip/wheels/f3/7c/ac/fcb6d867f806021c3730fd848970db988b1d0030b5d20c0e02
Successfully built Word2Vec


## Import Libraries

In [5]:
import pandas as pd
import networkx as nx
from node2vec import Node2Vec
from node2vec.edges import HadamardEmbedder
import pickle
import torch
import gensim
import numpy as np
import keras
from tensorflow import keras
import tensorflow as tf
from keras import backend as K
import os
from sklearn.decomposition import TruncatedSVD
from collections import defaultdict
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from sklearn.inspection import permutation_importance
from sklearn.model_selection import ParameterGrid
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn import metrics

## Read Data and Process Tuples

In [6]:
train_set = pd.read_csv('/content/gdrive/MyDrive/UniCredit/kg_train.csv')
val_set = pd.read_csv('/content/gdrive/MyDrive/UniCredit/kg_validation.csv')
test_set = pd.read_csv('/content/gdrive/MyDrive/UniCredit/kg_test_nolabel.csv')
train_val_set = pd.read_csv('/content/gdrive/MyDrive/UniCredit/kg_pykeen_trainvalidation.tsv', 
                            sep='\t', header=0, names=['source', 'relation', 'tail'])

In [7]:
train_set[['head', 'relation', 'tail']] = train_set['Triple'].str.split(' ', 2, expand=True)
val_set[['head', 'relation', 'tail']] = val_set['Triple'].str.split(' ', 2, expand=True)
test_set[['head', 'relation', 'tail']] = test_set['Triple'].str.split(' ', 2, expand=True)

In [ ]:
train_val_set.head()

,source,relation,tail
0,10005,tributariesLeft,10008
1,10005,tributariesLeft,10014
2,10005,tributariesLeft,10015
3,10005,tributariesLeft,10016
4,10005,tributariesRight,10007


## Generate Node Embeddings

Based on the paper: https://arxiv.org/abs/1607.00653
 the best feature embedding model for link prediction is Node2vec using HadamardEmbedder edge embedder.

In [ ]:
# Generate graph from dataframe
graph = nx.from_pandas_edgelist(train_val_set, source='source', 
                                target='tail', edge_attr='relation', 
                                create_using=nx.MultiDiGraph)

In [ ]:
# Initialize object for Node2Vec generation
node2vec = Node2Vec(graph, dimensions=200, 
                    walk_length=30, num_walks=100, workers=4)

In [ ]:
# Fit embedding model
embedding = node2vec.fit(window=10, min_count=1, batch_words=4)

In [10]:
embedding = gensim.models.KeyedVectors.load_word2vec_format('/content/gdrive/MyDrive/UniCredit/Copia di embedding_720_25')

In [11]:
# Object to generate relational edge embedding
edges_embs = HadamardEmbedder(keyed_vectors=embedding.wv)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


### Create triple tensor of dimensions (len(df), dim(emb), 3)

In [ ]:
A_list_train_val = []
for row in train_val_set.iterrows():
    source_emb = embedding[str(row[1][0])]
    tail_emb = embedding[str(row[1][2])]
    edge_emb = edges_embs[(str(row[1][0]), str(row[1][2]))]
    matrix_ = np.vstack((source_emb, edge_emb, tail_emb)).T
    A_list_train_val.append(matrix_)
A_train_val = np.stack(A_list_train_val)

In [12]:
A_list_train = []
for row in train_set.iterrows():
    string = row[1][1].split()
    source_emb = embedding[string[0]]
    tail_emb = embedding[string[2]]
    edge_emb = edges_embs[(string[0], string[2])]
    matrix_ = np.vstack((source_emb, edge_emb, tail_emb)).T
    A_list_train.append(matrix_)
A_train = np.stack(A_list_train)

In [13]:
A_list_test = []
for row in test_set.iterrows():
    string = row[1][1].split()
    source_emb = embedding[string[0]]
    tail_emb = embedding[string[2]]
    edge_emb = edges_embs[(string[0], string[2])]
    matrix_ = np.vstack((source_emb, edge_emb, tail_emb)).T
    A_list_test.append(matrix_)
A_test = np.stack(A_list_test)

In [14]:
A_list_val = []
for row in val_set.iterrows():
    string = row[1][1].split()
    source_emb = embedding[string[0]]
    tail_emb = embedding[string[2]]
    edge_emb = edges_embs[(string[0], string[2])]
    matrix_ = np.vstack((source_emb, edge_emb, tail_emb)).T
    A_list_val.append(matrix_)
A_val = np.stack(A_list_val)

### Store results into pickle file

In [15]:
#with open("/content/gdrive/MyDrive/UniCredit/A_train_val.txt", 
#          "wb") as fp:   #Pickling
#    pickle.dump(A_train_val, fp)

with open("/content/gdrive/MyDrive/UniCredit/A_test_700.txt", 
          "wb") as fp:   #Pickling
    pickle.dump(A_test, fp)

with open("/content/gdrive/MyDrive/UniCredit/A_train_700.txt", 
          "wb") as fp:   #Pickling
    pickle.dump(A_train, fp)

with open("/content/gdrive/MyDrive/UniCredit/A_val_700.txt", 
          "wb") as fp:   #Pickling
    pickle.dump(A_val, fp)

### Load Pickle files

In [16]:
#with open("/content/gdrive/MyDrive/UniCredit/A_train_val.txt", "rb") as fp:   # Unpickling
#    A_train_val = pickle.load(fp)

with open("/content/gdrive/MyDrive/UniCredit/A_test_700.txt", "rb") as fp:   # Unpickling
    A_test = pickle.load(fp)

with open("/content/gdrive/MyDrive/UniCredit/A_train_700.txt", "rb") as fp:   # Unpickling
    A_train = pickle.load(fp)

with open("/content/gdrive/MyDrive/UniCredit/A_val_700.txt", "rb") as fp:   # Unpickling
    A_val = pickle.load(fp)

### Melt tensor into a matrix of dimension (len(df), dim(emb)*3). Will then be used as features in the Embedders Model later on

In [17]:
A_train_melt = A_train.reshape(A_train.shape[0], A_train.shape[1]*3, order='F')
A_val_melt = A_val.reshape(A_val.shape[0], A_val.shape[1]*3, order='F')
A_test_melt = A_test.reshape(A_test.shape[0], A_test.shape[1]*3, order='F')

## CNN Model

Based on the paper: https://aclanthology.org/N18-2053.pdf
using a Convolutional Neural Network seems like a good idea for link prediction

In [ ]:
emb_dimension = A_test.shape[1]

class JSCNN_Factory:

  def __init__(self, emb_dimension, path='/content/gdrive/MyDrive/UniCredit'):
    self.inputs = keras.Input(shape=(emb_dimension, 3, 1)) # Embedding dimension, len((h,r,t))=3, 1 channel
    self.outputs = keras.layers.Dense(units=2, activation='softmax')
    self.config_file = open(os.path.join(path, 'conf_CNN.csv'), 'a')
    self.config_file.write('Id,number_conv_layers,number_dense_layers,number_channels_conv,number_channels_dense\n')
    self.model_id = 0


  def _clean(self):
    self.inputs = keras.Input(shape=(emb_dimension, 3, 1))
    self.outputs = keras.layers.Dense(units=2, activation='softmax')

  
  def _add_conv_module(self, number_channels_conv):
    self.hidden_layers = keras.layers.BatchNormalization()(self.hidden_layers)
    self.hidden_layers = keras.layers.Conv2D(filters=number_channels_conv, kernel_size=(1,3), padding='same', activation='elu')(self.hidden_layers)


  def _add_dense_module(self, number_channels_dense):
    self.hidden_layers = keras.layers.BatchNormalization()(self.hidden_layers)
    self.hidden_layers = keras.layers.Dense(units=number_channels_dense, activation='elu')(self.hidden_layers)

  
  def get_CNN(self, number_conv_layers, number_dense_layers, number_channels_conv, number_channels_dense):
    self.hidden_layers = self.inputs

    for _ in range(0, number_conv_layers):
      self._add_conv_module(number_channels_conv)
    
    self.hidden_layers = keras.layers.Flatten()(self.hidden_layers)
    self.hidden_layers = keras.layers.BatchNormalization()(self.hidden_layers)
    self.hidden_layers = keras.layers.Dropout(rate=0.2)(self.hidden_layers)

    for _ in range(0, number_dense_layers):
      self._add_dense_module(number_channels_dense)

    self.outputs = self.outputs(self.hidden_layers)

    self.model = keras.Model(inputs=self.inputs, outputs=self.outputs, name='JSCNN')

    self._save_model_conf(number_conv_layers, number_dense_layers, number_channels_conv, number_channels_dense)

    self._clean()
    
    return self.model


  def _save_model_conf(self, number_conv_layers, number_dense_layers, number_channels_conv, number_channels_dense):
    self.config_file.write(f'{self.model_id},{number_conv_layers},{number_dense_layers},{number_channels_conv},{number_channels_dense}\n')
    self.model_id += 1

In [ ]:
# Create object to build CNN
CNN_factory = JSCNN_Factory(emb_dimension)

In [ ]:
# Parametrize the CNN
model = CNN_factory.get_CNN(number_conv_layers=2, number_dense_layers=1, number_channels_conv=8, number_channels_dense=32)

In [ ]:
# CNN summary
model.summary()

Model: "JSCNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 700, 3, 1)]       0         
_________________________________________________________________
batch_normalization_24 (Batc (None, 700, 3, 1)         4         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 700, 3, 8)         32        
_________________________________________________________________
batch_normalization_25 (Batc (None, 700, 3, 8)         32        
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 700, 3, 8)         200       
_________________________________________________________________
flatten_6 (Flatten)          (None, 16800)             0         
_________________________________________________________________
batch_normalization_26 (Batc (None, 16800)             67200 

In [19]:
# Generate evaluation metrics for CNN
from keras import backend as K
import time

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
my_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

In [ ]:
# Compile CNN
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1_m])

In [ ]:
# Fit Model
model.fit(A_train, keras.utils.to_categorical(np.array(train_set.Label)), 
          validation_data=(A_val, keras.utils.to_categorical(np.array(val_set.Label))),
          epochs=50, batch_size=100, callbacks=my_callbacks)

In [20]:
# Select best epoch
model = tf.keras.models.load_model('/content/gdrive/MyDrive/UniCredit/Copia di model.89-0.27.h5', 
                                   custom_objects={'f1_m':f1_m})

In [ ]:
# Get CNN predictions
ynew_CNN = model.predict(A_test)

## Feature Reduction

We will also used the embeddings to train other classifiers such as Random Forest, Logisitic Regression, etc. but before that we will reduce the dimensionality

In [22]:
# SVD function
def apply_svd(data, n_components):
    
    svd = TruncatedSVD(n_components=n_components)
    svd.fit(data)
    transformed = svd.transform(data)

    return transformed

In [23]:
# Apply SVD on test, val and train data reducing to 15 components
n_components=15

list_feat = ['f_red'+ str(i) for i in range(15*3)]

red_val_e1 = pd.DataFrame(apply_svd(A_val_melt[:,:700], n_components), columns=list_feat[0:15])
red_val_r = pd.DataFrame(apply_svd(A_val_melt[:,700:1400], n_components), columns=list_feat[15:30])
red_val_e2 = pd.DataFrame(apply_svd(A_val_melt[:,1400:2100], n_components), columns=list_feat[30:45])

red_train_e1 = pd.DataFrame(apply_svd(A_train_melt[:,:700], n_components), columns=list_feat[0:15])
red_train_r = pd.DataFrame(apply_svd(A_train_melt[:,700:1400], n_components), columns=list_feat[15:30])
red_train_e2 = pd.DataFrame(apply_svd(A_train_melt[:,1400:2100], n_components), columns=list_feat[30:45])

red_test_e1 = pd.DataFrame(apply_svd(A_test_melt[:,:700], n_components), columns=list_feat[0:15])
red_test_r = pd.DataFrame(apply_svd(A_test_melt[:,700:1400], n_components), columns=list_feat[15:30])
red_test_e2 = pd.DataFrame(apply_svd(A_test_melt[:,1400:2100], n_components), columns=list_feat[30:45])

## Feature Generation

We will generate the following features:
- 6 features presented in the lab by UniCredit Lecturers
- CNN Prediction (0 or 1)
- CNN Prediction Probability [0, 1]
- 15*3 features from the original embeddings

In [24]:
def get_relations(df):
  """
  Get the relations in the graph as a dictionary of {'relation':[(head, tail), ...]}.

  :param df: pd.DataFrame containing the graph

  :return dictionary
  """
  pairs=defaultdict(list)
  relations=defaultdict(list)
  for i in df.iterrows():
      s = i[1]['head']
      t = i[1]['tail']
      r = i[1]['relation']
      pairs[(s,t)].append(r)
      relations[r].append((s,t))

  return relations

In [25]:
def compute_features(G,GUnd,GUndSimple,i,j,rel=None):
  """
  Function to compute the features of the graph

  :param G: MultiDiGraph
  :param GUnd: MultiGraph (undirected)
  :param GUndSimple: Simple undirected graph
  :param i: head node
  :param j: tail node
  :param rel: relation

  :return DataFrame with the features
  """
  cn=aa=jc=ind_i = outd_i = ind_j = outd_j =0
      
  if ((i in GUnd) and (j in GUnd)):
      cn = len(list(nx.common_neighbors(GUnd, i,j)))
  if ((i in GUndSimple) and (j in GUndSimple)):
      jc = [jc for u,v,jc in nx.jaccard_coefficient(GUndSimple, [(i,j)])][0]
  if (i in G):
      ind_i = G.in_degree(i)
      outd_i = G.out_degree(i)
  if (j in G):
      ind_j = G.in_degree(j)
      outd_j = G.out_degree(j)
  
  return cn,jc,ind_i,ind_j,outd_i,outd_j

In [26]:
#Given a triple, compute some node features
def compute_triples_features(G,GUnd,GUndSimple,rel,triple):
    curr_triples_feature={}
    curr_triples_feature['relation'] = rel
    curr_triples_feature['head'] = triple[0]
    curr_triples_feature['tail'] = triple[1]
    for ix,feature in enumerate(compute_features(G,GUnd,GUndSimple,triple[0],triple[1],rel)):
        curr_triples_feature['f'+str(ix)] = feature
    return curr_triples_feature

In [27]:
#Compute features for each positive and negative relation
def generate_df_features(G, relations):
    triples=[]
    GUnd = G.to_undirected()
    GUndSimple = nx.Graph(GUnd)
    for rel in relations.keys():
        for posTriple in relations[rel]:
            triples.append(compute_triples_features(G,GUnd,GUndSimple,rel,posTriple))
    df_rel = pd.DataFrame(triples)
    return df_rel

In [28]:
# Generate CNN features, prediction and probability
def get_CNN_pred(A, model):
  """
  Function to get the predictions of the CNN

  :param A: tensor [n_samples x embedding_len x 3]
  :param model: pre-trained CNN

  :return DataFrame with the predictions
  """
  ypred = model.predict(A)

  pred_classes = np.argmax(ypred, axis=1)
  pred_probs = np.max(ypred, axis=1)
  res = pd.DataFrame({'pred_class':pred_classes, 'pred_prob':pred_probs})

  return res

In [29]:
# Generate data frame with all the new features
def generate_new_data(G, data, A, model, test=False):
  """
  Generate the new fetures for each {train, val, test} set.

  :param G: graph
  :param data: DataFrame
  :param A: tensor of embeddings
  :param model: CNN
  :param test: boolean

  :return DataFrame
  """

  relations = get_relations(data)
  features = generate_df_features(G, relations)
  predictions_CNN = get_CNN_pred(A, model)
  features[['pred_class', 'pred_prob']] = predictions_CNN[['pred_class', 'pred_prob']]
  features['Id'] = data['Id']
  if not test:
    features['Label'] = data['Label']

  return features

In [30]:
Gtrain=nx.from_pandas_edgelist(train_set, source='head', target='tail', edge_attr='relation', create_using=nx.MultiDiGraph)
Gval = nx.from_pandas_edgelist(val_set, source='head', target='tail', edge_attr='relation', create_using=nx.MultiDiGraph)
Gtest = nx.from_pandas_edgelist(test_set, source='head', target='tail', edge_attr='relation', create_using=nx.MultiDiGraph)

In [31]:
new_data_train = generate_new_data(Gtrain, train_set, A_train, model)
new_data_val = generate_new_data(Gval, val_set, A_val, model)
new_data_test = generate_new_data(Gtest, test_set, A_test, model, test=True)

In [32]:
# This part includes the reduced embedding features
new_data_train = pd.concat((new_data_train, red_train_e1, 
                            red_train_r, red_train_e2), axis=1)
new_data_test = pd.concat((new_data_test, red_test_e1, 
                            red_test_r, red_test_e2), axis=1)
new_data_val = pd.concat((new_data_val, red_val_e1, 
                            red_val_r, red_val_e2), axis=1)

In [34]:
path_to_data = '/content/gdrive/MyDrive/UniCredit/'

new_data_train.to_csv(os.path.join(path_to_data, 'new_features_train.csv'), index=False)
new_data_val.to_csv(os.path.join(path_to_data, 'new_features_val.csv'), index=False)
new_data_test.to_csv(os.path.join(path_to_data, 'new_features_test.csv'), index=False)

In [35]:
new_data_test.head()

,relation,head,tail,f0,f1,f2,f3,f4,f5,pred_class,pred_prob,Id,f_red0,f_red1,f_red2,f_red3,f_red4,f_red5,f_red6,f_red7,f_red8,f_red9,f_red10,f_red11,f_red12,f_red13,f_red14,f_red15,f_red16,f_red17,f_red18,f_red19,f_red20,f_red21,f_red22,f_red23,f_red24,f_red25,f_red26,f_red27,f_red28,f_red29,f_red30,f_red31,f_red32,f_red33,f_red34,f_red35,f_red36,f_red37,f_red38,f_red39,f_red40,f_red41,f_red42,f_red43,f_red44
0,operatingSystem,1322,14477,0,0.0,0,7,1,0,0,1.000000,114754,2.101868,-0.079553,-0.225531,-0.118636,0.161321,-0.557222,0.392360,-0.926571,0.057648,0.341015,-0.156376,0.146295,0.708701,-0.181633,0.441124,0.187251,-0.045465,-0.041646,-0.038462,0.004715,0.011998,-0.079375,-0.017312,-0.052402,-0.124753,0.035394,-0.079712,-0.074768,0.011556,-0.022874,1.259990,0.132757,-0.112237,-0.015321,0.050881,0.004012,0.034895,-0.033267,-0.039484,-0.147908,-0.012306,-0.012360,0.027332,-0.093663,-0.047677
1,operatingSystem,30750,3366,0,0.0,0,6,1,0,0,1.000000,114755,2.186363,0.015829,0.505996,0.682810,-0.134592,-0.600302,-0.018313,-0.044109,-0.161858,0.141601,0.394762,0.118372,0.168606,-0.038113,-0.394330,0.199325,-0.026873,-0.101813,-0.089457,-0.165613,0.073620,-0.055757,-0.031126,-0.000560,0.020684,-0.001004,-0.122088,-0.033235,0.012096,0.007523,0.995772,0.095329,-0.063124,-0.035066,-0.014856,0.000628,-0.003043,0.011868,-0.028059,-0.009441,-0.023481,0.018329,-0.024700,-0.005727,0.002150
2,operatingSystem,33045,932,0,0.0,0,1,6,0,0,0.943398,114756,1.077688,0.014126,0.017178,0.014746,0.011253,0.001753,0.076797,0.019125,0.017265,0.039135,-0.014828,0.038694,-0.027824,-0.033979,-0.020424,0.090825,-0.016433,-0.044762,-0.040702,-0.032906,0.001551,-0.013504,0.008466,0.017054,-0.008003,-0.006438,-0.015324,0.000127,0.007130,0.005139,1.031989,0.001647,0.009557,-0.066868,0.117456,0.067883,0.005624,0.083670,-0.041310,0.019223,0.058705,-0.080245,0.011256,-0.047343,0.007684
3,operatingSystem,42206,5554,0,0.0,0,1,3,0,1,0.647623,114757,0.967985,0.063208,0.022269,0.000841,-0.018002,0.019148,0.054580,-0.001534,-0.010605,-0.087606,0.026118,0.018361,0.032771,0.044465,-0.000250,0.132729,-0.006361,-0.035315,-0.037589,-0.031638,0.014176,-0.004402,0.019464,0.003981,0.012548,0.012355,-0.019893,-0.005656,0.011443,-0.003142,1.795733,0.205193,0.075002,-0.012218,0.002310,0.091902,-0.312525,-0.051197,-0.009161,-0.031660,0.082858,0.062917,0.044360,0.130582,0.046311
4,operatingSystem,38566,4745,0,0.0,0,2,2,0,1,0.786682,114758,1.318649,-0.217600,-0.202893,0.055080,-0.154152,0.001707,-0.009828,-0.049747,0.011008,-0.025078,0.046389,-0.041461,-0.056684,0.073555,0.026354,0.414417,-0.149934,0.014160,-0.102621,0.059356,0.016070,-0.035764,0.023693,-0.071696,0.033463,0.042936,0.022403,0.003367,0.027534,0.007439,4.725959,-1.370295,0.848810,-0.877565,0.598260,-0.486537,-0.885893,0.399028,1.154598,1.389623,1.660212,0.405414,-1.478132,0.633046,-0.178540


## Ensemble Method

We will apply 5 different models and choose the prediction label that gets the highest number of votes

In [3]:
train_df = pd.read_csv('/content/gdrive/MyDrive/UniCredit/new_features_train.csv')
val_df = pd.read_csv('/content/gdrive/MyDrive/UniCredit/new_features_val.csv')
test_df = pd.read_csv('/content/gdrive/MyDrive/UniCredit/new_features_test.csv')

In [4]:
# Random Forest
def train_model_parameter_RF(df_train,df_validation,param, 
                             features=['f0','f1','f2','f3','f4','f5'], test=False):
    y_train = df_train['Label']
    X_train = df_train[features]
    if not test:
        y_validation = df_validation['Label']
    X_validation = df_validation[features]
    model = RandomForestClassifier(**param, random_state=42)
    model.fit(X_train, y_train)
    pred_validation = model.predict_proba(X_validation)
    pred_class_validation = model.predict(X_validation)
    if not test:
        fpr, tpr, thresholds = metrics.roc_curve(y_validation, pred_validation[:,0], pos_label=1)
        tn, fp, fn, tp = metrics.confusion_matrix(y_validation,pred_class_validation).ravel()
    if not test:
        return {'train':y_train.shape[0],'test':y_validation.shape[0],
                'tn':tn,'fp':fp,'fn':fn,'tp':tp,
                'AUC':metrics.auc(fpr, tpr),'precision':metrics.precision_score(y_validation,pred_class_validation),
                'recall':metrics.recall_score(y_validation,pred_class_validation),
                'accuracy':metrics.accuracy_score(y_validation,pred_class_validation),
                'F1':metrics.f1_score(y_validation,pred_class_validation),
                'Prediction':pred_class_validation}
    else:
        return {'Prediction':pred_class_validation}

In [5]:
# Random Forest Parameter Grid
param_grid_RF = {
    'max_depth': [10, 20, 30],
    'n_estimators': [100, 150, 200]
}
param_grid_RF_list = list(ParameterGrid(param_grid_RF))

In [6]:
# SVM
def train_model_parameter_SV(df_train,df_validation,param, 
                             features=['f0','f1','f2','f3','f4','f5'], 
                             max_iter=5000, test=False):
    y_train = df_train['Label']
    X_train = df_train[features]
    y_validation = df_validation['Label']
    X_validation = df_validation[features]
    model = SVC(**param, random_state=42, max_iter=max_iter)
    model.fit(X_train, y_train)
    pred_validation = model.predict_proba(X_validation)
    pred_class_validation = model.predict(X_validation)
    fpr, tpr, thresholds = metrics.roc_curve(y_validation, pred_validation[:,0], pos_label=1)
    tn, fp, fn, tp = metrics.confusion_matrix(y_validation,pred_class_validation).ravel()
    return {'train':y_train.shape[0],'test':y_validation.shape[0],
            'tn':tn,'fp':fp,'fn':fn,'tp':tp,
            'AUC':metrics.auc(fpr, tpr),'precision':metrics.precision_score(y_validation,pred_class_validation),
            'recall':metrics.recall_score(y_validation,pred_class_validation),
            'accuracy':metrics.accuracy_score(y_validation,pred_class_validation),
            'F1':metrics.f1_score(y_validation,pred_class_validation),
            'Prediction':pred_class_validation}

In [7]:
# SVM Parameter Grid
param_grid_SV = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
}
param_grid_SV_list = list(ParameterGrid(param_grid_SV))

In [8]:
# Linear SVM
def train_model_parameter_LS(df_train,df_validation,param, 
                                 features=['f0','f1','f2','f3','f4','f5'], test=False):
    y_train = df_train['Label']
    X_train = df_train[features]
    if not test:
        y_validation = df_validation['Label']
    X_validation = df_validation[features]
    model = LinearSVC(**param, random_state=42)
    model.fit(X_train, y_train)
    #pred_validation = model.predict_proba(X_validation)
    pred_class_validation = model.predict(X_validation)
    if not test:
        #fpr, tpr, thresholds = metrics.roc_curve(y_validation, pred_validation[:,0], pos_label=1)
        tn, fp, fn, tp = metrics.confusion_matrix(y_validation,pred_class_validation).ravel()
    if not test:
        return {'train':y_train.shape[0],'test':y_validation.shape[0],
                'tn':tn,'fp':fp,'fn':fn,'tp':tp,
                #'AUC':metrics.auc(fpr, tpr),
                'precision':metrics.precision_score(y_validation,pred_class_validation),
                'recall':metrics.recall_score(y_validation,pred_class_validation),
                'accuracy':metrics.accuracy_score(y_validation,pred_class_validation),
                'F1':metrics.f1_score(y_validation,pred_class_validation),
                'Prediction':pred_class_validation}
    else:
        return {'Prediction':pred_class_validation}

In [9]:
# Linear SVM Parameter Grid
param_grid_LS = {
    'penalty': ['l1', 'l2'],
    'loss': ['hinge', 'squared_hinge'],
    'C': [0.001, .1, 5],
}
param_grid_LS_list = list(ParameterGrid(param_grid_LS))

In [10]:
# XGBoost
def train_model_parameter_XG(df_train,df_validation,param, 
                                 features=['f0','f1','f2','f3','f4','f5'], test=False):
    y_train = df_train['Label']
    X_train = df_train[features]
    if not test:
        y_validation = df_validation['Label']
    X_validation = df_validation[features]
    model = XGBClassifier(**param, random_state = 42)
    model.fit(X_train, y_train)
    pred_validation = model.predict_proba(X_validation)
    pred_class_validation = model.predict(X_validation)
    if not test:
        fpr, tpr, thresholds = metrics.roc_curve(y_validation, pred_validation[:,0], pos_label=1)
        tn, fp, fn, tp = metrics.confusion_matrix(y_validation,pred_class_validation).ravel()
    if not test:
        return {'train':y_train.shape[0],'test':y_validation.shape[0],
                'tn':tn,'fp':fp,'fn':fn,'tp':tp,
                'AUC':metrics.auc(fpr, tpr),'precision':metrics.precision_score(y_validation,pred_class_validation),
                'recall':metrics.recall_score(y_validation,pred_class_validation),
                'accuracy':metrics.accuracy_score(y_validation,pred_class_validation),
                'F1':metrics.f1_score(y_validation,pred_class_validation),
                'Prediction':pred_class_validation}
    else:
        return {'Prediction':pred_class_validation}

In [11]:
# XGBoost Parameter Grid
param_grid_XG = {'max_depth':[3, 6, 9],
              'min_child_weight':[1, 6],
              'learning_rate': [0.0001, 0.1]}
param_grid_XG_list = list(ParameterGrid(param_grid_XG))

In [12]:
# Logistic Regression
def train_model_parameter_LR(df_train, df_validation, param, 
                             features=['f0','f1','f2','f3','f4','f5'], test=False):
    y_train = df_train['Label']
    X_train = df_train[features]
    if not test:
        y_validation = df_validation['Label']
    X_validation = df_validation[features]
    model = LogisticRegression(**param, random_state = 42)
    model.fit(X_train, y_train)
    pred_validation = model.predict_proba(X_validation)
    pred_class_validation = model.predict(X_validation)
    if not test:
        fpr, tpr, thresholds = metrics.roc_curve(y_validation, pred_validation[:,0], pos_label=1)
        tn, fp, fn, tp = metrics.confusion_matrix(y_validation,pred_class_validation).ravel()
    if not test:
        return {'train':y_train.shape[0],'test':y_validation.shape[0],
                'tn':tn,'fp':fp,'fn':fn,'tp':tp,
                'AUC':metrics.auc(fpr, tpr),
                'precision':metrics.precision_score(y_validation,pred_class_validation),
                'recall':metrics.recall_score(y_validation,pred_class_validation),
                'accuracy':metrics.accuracy_score(y_validation,pred_class_validation),
                'F1':metrics.f1_score(y_validation,pred_class_validation),
                'Prediction':pred_class_validation}
    else:
        return {'Prediction':pred_class_validation}

In [13]:
# Logistic Regression Parameter Grid
param_grid_LR = {'penalty':['l1', 'l2', 'elasticnet', 'none'],
              'C':[0.01, 0.5, 1],
              'solver': ['saga'],
              'max_iter': [5000]}
param_grid_LR_list = list(ParameterGrid(param_grid_LR))



In [ ]:
params = {'LR':param_grid_LR_list, 
          'XG':param_grid_XG_list, 
          'LS':param_grid_LS_list, 
          'SV':param_grid_SV_list,
          'RF':param_grid_RF_list}

In [ ]:
# Hyper-parametrization of all models
features = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5'] + ['f_red'+ str(i) for i in range(15*3)]
X_train = train_df
X_test = val_df
score_param = []
for model_param in params:
    print('============')
    print('New Model:', model_param)
    print('============')
    for i in params[model_param]:
        print(i)
        if model_param == 'LR':
            try:
                res = train_model_parameter_LR(df_train=X_train, 
                                                df_validation=X_test,
                                                param=i,
                                                features=features)
            except:
                print('Something went wrong')
                continue
        if model_param == 'XG':
            try:
                res = train_model_parameter_XG(df_train=X_train, 
                                                df_validation=X_test,
                                                param=i,
                                                features=features)
            except:
                print('Something went wrong')
                continue
        if model_param == 'LS':
            try:
                res = train_model_parameter_LS(df_train=X_train, 
                                                df_validation=X_test,
                                                param=i,
                                                features=features)
            except:
                print('Something went wrong')
                continue
        if model_param == 'SV':
            try:
                res = train_model_parameter_SV(df_train=X_train, 
                                                df_validation=X_test,
                                                param=i,
                                                features=features, 
                                                max_iter=100000)
            except:
                print('Something went wrong')
                continue
        if model_param == 'RF':
            try:
                res = train_model_parameter_RF(df_train=X_train, 
                                                df_validation=X_test,
                                                param=i,
                                                features=features)
            except:
                print('Something went wrong')
                continue
        score_param.append((model_param, i, res['F1']))

New Model: LR
{'C': 0.01, 'max_iter': 5000, 'penalty': 'l1', 'solver': 'saga'}
{'C': 0.01, 'max_iter': 5000, 'penalty': 'l2', 'solver': 'saga'}
{'C': 0.01, 'max_iter': 5000, 'penalty': 'elasticnet', 'solver': 'saga'}
Something went wrong
{'C': 0.01, 'max_iter': 5000, 'penalty': 'none', 'solver': 'saga'}


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


{'C': 0.5, 'max_iter': 5000, 'penalty': 'l1', 'solver': 'saga'}
{'C': 0.5, 'max_iter': 5000, 'penalty': 'l2', 'solver': 'saga'}
{'C': 0.5, 'max_iter': 5000, 'penalty': 'elasticnet', 'solver': 'saga'}
Something went wrong
{'C': 0.5, 'max_iter': 5000, 'penalty': 'none', 'solver': 'saga'}


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


{'C': 1, 'max_iter': 5000, 'penalty': 'l1', 'solver': 'saga'}
{'C': 1, 'max_iter': 5000, 'penalty': 'l2', 'solver': 'saga'}
{'C': 1, 'max_iter': 5000, 'penalty': 'elasticnet', 'solver': 'saga'}
Something went wrong
{'C': 1, 'max_iter': 5000, 'penalty': 'none', 'solver': 'saga'}
New Model: XG
{'learning_rate': 0.0001, 'max_depth': 3, 'min_child_weight': 1}
{'learning_rate': 0.0001, 'max_depth': 3, 'min_child_weight': 6}
{'learning_rate': 0.0001, 'max_depth': 6, 'min_child_weight': 1}
{'learning_rate': 0.0001, 'max_depth': 6, 'min_child_weight': 6}
{'learning_rate': 0.0001, 'max_depth': 9, 'min_child_weight': 1}
{'learning_rate': 0.0001, 'max_depth': 9, 'min_child_weight': 6}
{'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1}
{'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 6}
{'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 1}
{'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 6}
{'learning_rate': 0.1, 'max_depth': 9, 'min_child_weight': 1}
{'learn

...

{'C': 5, 'loss': 'hinge', 'penalty': 'l1'}
Something went wrong
{'C': 5, 'loss': 'hinge', 'penalty': 'l2'}


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


{'C': 5, 'loss': 'squared_hinge', 'penalty': 'l1'}
Something went wrong
{'C': 5, 'loss': 'squared_hinge', 'penalty': 'l2'}


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


New Model: RF
{'max_depth': 10, 'n_estimators': 100}
{'max_depth': 10, 'n_estimators': 150}
{'max_depth': 10, 'n_estimators': 200}
{'max_depth': 20, 'n_estimators': 100}
{'max_depth': 20, 'n_estimators': 150}
{'max_depth': 20, 'n_estimators': 200}
{'max_depth': 30, 'n_estimators': 100}
{'max_depth': 30, 'n_estimators': 150}
{'max_depth': 30, 'n_estimators': 200}


In [ ]:
score_df = pd.DataFrame(score_param, columns=['Model', 'Params', 'F1'])

In [ ]:
score_df

,Model,Params,F1
0,LR,"{'C': 0.01, 'max_iter': 5000, 'penalty': 'l1',...",0.659672
1,LR,"{'C': 0.01, 'max_iter': 5000, 'penalty': 'l2',...",0.659995
2,LR,"{'C': 0.01, 'max_iter': 5000, 'penalty': 'none...",0.587129
3,LR,"{'C': 0.5, 'max_iter': 5000, 'penalty': 'l1', ...",0.598032
4,LR,"{'C': 0.5, 'max_iter': 5000, 'penalty': 'l2', ...",0.595990
5,LR,"{'C': 0.5, 'max_iter': 5000, 'penalty': 'none'...",0.587129
6,LR,"{'C': 1, 'max_iter': 5000, 'penalty': 'l1', 's...",0.592129
7,LR,"{'C': 1, 'max_iter': 5000, 'penalty': 'l2', 's...",0.592383
8,LR,"{'C': 1, 'max_iter': 5000, 'penalty': 'none', ...",0.587129
9,XG,"{'learning_rate': 0.0001, 'max_depth': 3, 'min...",0.740223


In [ ]:
score_df.loc[score_df.groupby("Model")["F1"].idxmax()][['Model', 'Params']]

,Model,Params
1,LR,"{'C': 0.01, 'max_iter': 5000, 'penalty': 'l2',..."
22,LS,"{'C': 0.001, 'loss': 'squared_hinge', 'penalty..."
28,RF,"{'max_depth': 10, 'n_estimators': 150}"
16,XG,"{'learning_rate': 0.1, 'max_depth': 3, 'min_ch..."


In [ ]:
X_train = train_df
X_test = test_df
pred_dict = {}
features = ['f0', 'f1', 'f2', 'f3', 
            'f4', 'f5', 
            'pred_class', 
            'pred_prob'] + ['f_red'+ str(i) for i in range(15*3)]

for model_param, i in temp:

    print(i)
    print(model_param)

    if model_param == 'LR':
        try:
            res = train_model_parameter_LR(df_train=X_train, 
                                            df_validation=X_test,
                                            param=i,
                                            features=features, test=True)
        except:
            print('Something went wrong')
            continue
    if model_param == 'XG':
        try:
            res = train_model_parameter_XG(df_train=X_train, 
                                            df_validation=X_test,
                                            param=i,
                                            features=features, test=True)
        except:
            print('Something went wrong')
            continue
    if model_param == 'LS':
        try:
            res = train_model_parameter_LS(df_train=X_train, 
                                            df_validation=X_test,
                                            param=i,
                                            features=features, test=True)
        except:
            print('Something went wrong')
            continue
    if model_param == 'SV':
        try:
            res = train_model_parameter_SV(df_train=X_train, 
                                            df_validation=X_test,
                                            param=i,
                                            features=features, 
                                            max_iter=100000, test=True)
        except:
            print('Something went wrong')
            continue
    if model_param == 'RF':
        try:
            res = train_model_parameter_RF(df_train=X_train, 
                                            df_validation=X_test,
                                            param=i,
                                            features=features, test=True)
        except:
            print('Something went wrong')
            continue
    pred_dict[model_param] = res['Prediction']



In [70]:
pred_df = pd.DataFrame(pred_dict)

In [71]:
pred_df.shape

(28690, 4)

In [27]:
pred_df.head()

,LR,LS,RF,XG
0,1,1,0,1
1,1,1,0,0
2,1,0,1,1
3,1,1,0,0
4,0,0,0,0


In [62]:
merged_df = pd.concat((test_df, pred_df), axis=1)

In [72]:
merged_df['Predicted'] = merged_df.apply(lambda x: int(sum([x['RF'], x['XG'], x['pred_class']]) >= 2), axis=1)

In [65]:
final_prediction = merged_df

## Save final prediction

In [73]:
final_prediction[['Id', 'Predicted']].to_csv('final_pred.csv', index=False)

In [74]:
final_prediction.shape

(28690, 63)